In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# rating_final.csv
url = 'https://drive.google.com/file/d/1ptu4AlEXO4qQ8GytxKHoeuS1y4l_zWkC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
frame = pd.read_csv(path)

# 'geoplaces2.csv'
url = 'https://drive.google.com/file/d/1ee3ib7LqGsMUksY68SD9yBItRvTFELxo/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
geodata = pd.read_csv(path, encoding = 'CP1252') # change encoding to 'mbcs' in Windows

places = geodata[['placeID', 'name']]

users_items = pd.pivot_table(data=frame, 
                                 values='rating', 
                                 index='userID', 
                                 columns='placeID')

users_items.fillna(0, inplace=True)

user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)

In [ ]:
def weighted_user_rec(user_id, n):
  weights = (user_similarities.query("userID!=@user_id")[user_id] / sum(user_similarities.query("userID!=@user_id")[user_id]))
  not_visited_restaurants = users_items.loc[users_items.index!=user_id, users_items.loc[user_id,:]==0]
  weighted_averages = pd.DataFrame(not_visited_restaurants.T.dot(weights), columns=["predicted_rating"])
  recommendations = weighted_averages.merge(places, left_index=True, right_on="placeID")
  top_recommendations = recommendations.sort_values("predicted_rating", ascending=False).head(n)
  return top_recommendations

In [ ]:
def chat_bot():
    print("Hi! I'm your personal recommender. Tell me your userID.")
    user_id = input()
    recom = weighted_user_rec(user_id, 1)
    print(f"You will probably like the restaurant: {list(recom['name'])[0]}")
    
chat_bot()

Hi! I'm your personal recommender. Tell me your userID.
U1077
You will probably like the restaurant: La Cantina Restaurante


Make your a chatbot for your movie recommender but make it smarter:
- Give an error message if the user enters unrecognised information
- Account for user errors such as capital letters in the wrong places, or the word `the` at the end of movies (e.g. Princess Bride, the)
- Ask the user for clarification if your search brings back multiple results. For example, there are lots of movies with the word 'Thing' in the title, including 2 movies called 'Thing, the', your chatbot should ask the user which of these movies they meant
- Create a FAQ dictionary to answer typical chatbot questions such as 'What is your name?' or 'Tell me a joke'